In [19]:
import pandas as pd
import datetime as dt
import numpy as np
from collections import OrderedDict

### Read Data

In [13]:
import pandas as pd
file = "SuperstoreTransaction.csv"
data = pd.read_csv(file, sep=",", header=0)
data.head()

Row ID        Order ID  Order Date   Ship Date       Ship Mode Customer ID  \
0       1  CA-2016-152156   11/8/2016  11/11/2016    Second Class    CG-12520   
1       2  CA-2016-152156   11/8/2016  11/11/2016    Second Class    CG-12520   
2       3  CA-2016-138688   6/12/2016   6/16/2016    Second Class    DV-13045   
3       4  US-2015-108966  10/11/2015  10/18/2015  Standard Class    SO-20335   
4       5  US-2015-108966  10/11/2015  10/18/2015  Standard Class    SO-20335   

     Customer Name    Segment        Country             City    ...     \
0      Claire Gute   Consumer  United States        Henderson    ...      
1      Claire Gute   Consumer  United States        Henderson    ...      
2  Darrin Van Huff  Corporate  United States      Los Angeles    ...      
3   Sean O'Donnell   Consumer  United States  Fort Lauderdale    ...      
4   Sean O'Donnell   Consumer  United States  Fort Lauderdale    ...      

  Postal Code  Region       Product ID         Category Sub-Category  \
0       42420   South  FUR-BO-10001798        Furniture    Bookcases   
1       42420   South  FUR-CH-10000454        Furniture       Chairs   
2       90036    West  OFF-LA-10000240  Office Supplies       Labels   
3       33311   South  FUR-TA-10000577        Furniture       Tables   
4       33311   South  OFF-ST-10000760  Office Supplies      Storage   

                                        Product Name     Sales  Quantity  \
0                  Bush Somerset Collection Bookcase  261.9600         2   
1  Hon Deluxe Fabric Upholstered Stacking Chairs,...  731.9400         3   
2  Self-Adhesive Address Labels for Typewriters b...   14.6200         2   
3      Bretford CR4500 Series Slim Rectangular Table  957.5775         5   
4                     Eldon Fold 'N Roll Cart System   22.3680         2   

   Discount    Profit  
0      0.00   41.9136  
1      0.00  219.5820  
2      0.00    6.8714  
3      0.45 -383.0310  
4      0.20    2.5164  

[5 rows x 21 columns]

### Understand dataset

In [21]:
#Check df length
len(data)

9994

In [22]:
#Check column names
data.columns

Index(['Order ID', 'Order Date', 'Customer ID', 'Customer Name', 'Product ID',
       'Product Name', 'Sales', 'Quantity'],
      dtype='object')

In [24]:
data.describe()

Sales     Quantity
count   9994.000000  9994.000000
mean     229.858001     3.789574
std      623.245101     2.225110
min        0.444000     1.000000
25%       17.280000     2.000000
50%       54.490000     3.000000
75%      209.940000     5.000000
max    22638.480000    14.000000

### Drop unnecessary columns

In [17]:
data = data.drop(['Row ID','Ship Date', 'Ship Mode','Segment', 'Country', 'City', 'State',
       'Postal Code', 'Region', 'Discount', 'Profit', 'Category', 'Sub-Category'], axis =1)

### Convert Timestamp to Datetime Object

In [18]:
#check column data types
data.dtypes

Order ID          object
Order Date        object
Customer ID       object
Customer Name     object
Product ID        object
Product Name      object
Sales            float64
Quantity           int64
dtype: object

In [20]:
data["Order Date"] = pd.to_datetime(data["Order Date"], format='%m/%d/%Y')
data.head()
print("Minimal Date=%s, Maximal Date=%s"%(min(data["Order Date"]).strftime("%Y-%m-%d %H:%M"), \
                                          max(data["Order Date"]).strftime("%Y-%m-%d %H:%M")))



Minimal Date=2014-01-03 00:00, Maximal Date=2017-12-30 00:00


### Calculate RFM Features for Users at Each Checkpoint

In [30]:
Start_Date_Obj = dt.datetime.strptime("4/1/2014", "%m/%d/%Y")
End_Date_Obj = dt.datetime.strptime("5/1/2014", "%m/%d/%Y")
Time_Window = 120 #days. Only consider customers who have activities within the recent 60 days
FM_Window = 30 #days for frequency and monetary

check_point_date = Start_Date_Obj
UserID = []
Checkpoint = []
Recency = []
Frequency = []
Monetary_Value = []
Monetary_Quantity = []
while check_point_date <= End_Date_Obj:
    window_start = check_point_date - dt.timedelta(days = Time_Window)
    mask = (data["Order Date"] >= window_start) & (data["Order Date"] < check_point_date)
    # Get the data in [checkpoint-60days, checkpoint]
    data_checkpoint = data.loc[mask]
    # Get the ids of users who have activities in [checkpoint-60days, checkpoint]
    unique_users = list(set(data_checkpoint["Customer ID"]))
    print("There are %d unique users."%(len(unique_users)))
    FM_Window_Start = check_point_date - dt.timedelta(days = FM_Window)
    for user in unique_users:
        UserID.append(user)
        Checkpoint.append(check_point_date)
        mask = data_checkpoint["Customer ID"] == user
        data_checkpoint_user = data_checkpoint.loc[mask]
        delta = check_point_date - max(data_checkpoint_user["Order Date"])
        recency = delta.days #Recency, days between checkpoint and last transaction time
        mask = data_checkpoint_user["Order Date"] >= FM_Window_Start
        data_checkpoint_user_fm = data_checkpoint_user.loc[mask]
        frequency = data_checkpoint_user_fm.shape[0]
        value = np.sum(data_checkpoint_user_fm.iloc[:, 6]) #monetary values
        quantity = np.sum(data_checkpoint_user_fm.iloc[:, 7])#monetary quantity
        Recency.append(recency)
        Frequency.append(frequency)
        Monetary_Value.append(value)
        Monetary_Quantity.append(quantity)
    check_point_date = check_point_date + dt.timedelta(days = 1)
# Consolidate all columns into a signle data frame
RFM_Dict = OrderedDict([ ('UserID', UserID),
          ('Checkpoint', Checkpoint),
          ('Recency',  Recency),
          ('Frequency', Frequency),
          ('Value', Monetary_Value),
          ('Quantity', Monetary_Quantity)] )
RFM_Frame = pd.DataFrame.from_dict(RFM_Dict)
RFM_Frame.head()


There are 121 unique users.
There are 122 unique users.
There are 124 unique users.
There are 125 unique users.
There are 128 unique users.
There are 133 unique users.
There are 137 unique users.
There are 140 unique users.
There are 144 unique users.
There are 144 unique users.
There are 144 unique users.
There are 148 unique users.
There are 151 unique users.
There are 152 unique users.
There are 152 unique users.
There are 153 unique users.
There are 154 unique users.
There are 154 unique users.
There are 155 unique users.
There are 157 unique users.
There are 158 unique users.
There are 160 unique users.
There are 161 unique users.
There are 163 unique users.
There are 163 unique users.
There are 166 unique users.
There are 168 unique users.
There are 168 unique users.
There are 172 unique users.
There are 176 unique users.
There are 177 unique users.


UserID Checkpoint  Recency  Frequency     Value  Quantity
0  NF-18475 2014-04-01       18          1    91.056         6
1  KA-16525 2014-04-01       21          1   108.920        14
2  BS-11590 2014-04-01       77          0     0.000         0
3  SC-20050 2014-04-01       14          3  3247.158        13
4  BD-11500 2014-04-01       59          0     0.000         0